In [1]:
import torch
from torch.autograd import Variable

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [4]:
import pandas as pd

In [5]:
wine = load_wine()

In [6]:
for key in wine.keys():
    print(key)

data
target
frame
target_names
DESCR
feature_names


In [7]:
df = pd.DataFrame(wine.data, columns=wine['feature_names'])

In [8]:
df.head(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0


In [9]:
train_X, test_X, train_Y, test_Y = train_test_split(wine['data'], wine['target'], test_size=0.2)

In [10]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

In [11]:
train_X.shape

torch.Size([142, 13])

In [12]:
wine['data'].shape

(178, 13)

In [13]:
train = TensorDataset(train_X, train_Y)
test = TensorDataset(test_X, test_Y)

train_loader = DataLoader(train, batch_size = 16, shuffle = True)
test_loader = DataLoader(test, batch_size = len(test), shuffle = True)

In [14]:
class WineClassifier(nn.Module):
    def __init__(self):
        super(WineClassifier,self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(13,96),
            nn.ReLU(),
            nn.Linear(96,3)
        )
    def forward(self,x):
        return self.classifier(x)

In [15]:
net = WineClassifier()

In [16]:
net

WineClassifier(
  (classifier): Sequential(
    (0): Linear(in_features=13, out_features=96, bias=True)
    (1): ReLU()
    (2): Linear(in_features=96, out_features=3, bias=True)
  )
)

In [17]:
net(torch.rand(3,13))

tensor([[ 0.1335, -0.0963,  0.0399],
        [ 0.2768, -0.0482,  0.1083],
        [ 0.1206, -0.0341,  0.0799]], grad_fn=<AddmmBackward>)

In [18]:
def testset_evaluation():
    correct = 0 
    total = 0

    for test_x, test_y in test_loader:
        result = torch.max(net(test_x).data, 1)[1]
        correct = correct + sum(test_y.data.numpy() == result.numpy())
        total = total + len(test_x)

    print(correct/total)

In [19]:
optimizer = optim.Adam(net.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    total_loss = 0
    for train_x, train_y in train_loader:
        optimizer.zero_grad()
        output = net(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss = total_loss + loss.data
    print(total_loss)
    testset_evaluation()

tensor(56.7149)
0.3611111111111111
tensor(16.5021)
0.5833333333333334
tensor(13.3810)
0.5833333333333334
tensor(9.5547)
0.5277777777777778
tensor(7.2270)
0.5555555555555556
tensor(6.0471)
0.5833333333333334
tensor(5.3160)
0.6388888888888888
tensor(5.1255)
0.6944444444444444
tensor(5.9309)
0.7222222222222222
tensor(4.5618)
0.6388888888888888
